In [1]:
import numpy as np
import sympy as sp

sp.init_printing()
t1, t2 = sp.symbols('t1 t2')
def RX(params):
  params =sp.symbols(params,real=True,)
  _RX = np.array([[sp.cos(params/2),-sp.I*sp.sin(params/2)],[-sp.I*sp.sin(params/2),sp.cos(params/2)]])
  return _RX

def RY(params):
  params =sp.symbols(params ,real=True,)
  _RY = np.array([[sp.cos(params/2),-1*sp.sin(params/2)],[sp.sin(params/2),sp.cos(params/2)]])
  return _RY


def Z_3(s='IZI'):
  m_dict = {
    'Z' : np.array([[1,0],[0,-1]]),
    'I' : np.eye(2)
  }
  result = np.kron(np.kron(m_dict[s[0]], m_dict[s[1]]), m_dict[s[2]])
  return result

def pqc(layer):
  C_NOT =np.array([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]])

  I_CNOT= np.kron(np.eye(2),C_NOT)
  CNOT_I= np.kron(C_NOT,np.eye(2))
  angle_embeding = np.kron(np.kron(RX('x1'),RX('x2')),np.eye(2))
  cnot_layer = I_CNOT @CNOT_I
  for i in range(layer):
    tmp=np.eye(8)
    pqc_layer =np.kron(np.kron(RY(f't{i+1}'),RY(f't{i+2}')),RY(f't{i+3}'))
    tmp = tmp@pqc_layer@cnot_layer
  tmp =tmp@cnot_layer@angle_embeding
  return tmp

In [2]:
ans =sp.trigsimp(pqc(3))

In [3]:
x = np.array([1,0,0,0,0,0,0,0])
x= x.reshape(8,1)
res= ans@x

In [ ]:
x1= res.T@Z_3("ZII")@res
sp.simplify(x1)

In [ ]:
dev = qml.device("default.qubit", wires=3)
class QModel(nn.Module):

    def __init__(self, 3):
        super(QModel, self).__init__()
        # Initialize trainable parameters (theta) for the quantum circuit
        self.n_wires = 3
        self.n_layers = 3
        self.n_qubit = self.n_wires
        self.theta = nn.Parameter(
            torch.rand(self.n_layers * self.n_wires * 2, dtype=torch.float32),
            requires_grad=True,
        )  # 9 trainable parameters

    @qml.qnode(dev, interface="torch")  # , diff_method="parameter-shift")
    def quantum_circuit_n_qubit(self, x, theta):
        # Encoding circuit: rotate qubits based on input data (x, y)
        qml.AngleEmbedding(x, wires=[0, 1], rotation="X")
        # Parameterized quantum circuit with entanglement
        for layer_count in range(self.n_layers):
            for i in range(self.n_wires - 1):
                qml.CNOT(wires=[i, i + 1])

            for i in range(self.n_wires):
                qml.RX(theta[layer_count * self.n_wires + i * 2 + 1], wires=i)

            for i in range(self.n_wires - 1):
                qml.CNOT(wires=[i, i + 1])

            for i in range(self.n_wires):
                qml.RY(theta[layer_count * self.n_wires + i * 2], wires=i)


        return [qml.expval(qml.PauliZ(i)) for i in range(3)]

    def forward(self, x):
        return F.relu(
            torch.stack((self.quantum_circuit_n_qubit(self, x=x, theta=self.theta))).T
        )

In [ ]:
x2= res.T@Z_3("IZI")@res
sp.trigsimp(x2)

In [25]:
x3= res.T@Z_3("IIZ")@res
sp.trigsimp(x3)